In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 74.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import math

from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import pipeline

In [ ]:
#df = pd.read_excel('leaders_bert.xlsx')
df = pd.read_excel('/content/drive/MyDrive/MSc/MAKATÖÖ/leaders_bert.xlsx')
df.head()

,nimi,erakond,kuupäev,postitus,clean_text,estnltk_text,lemma_text,bert_text
0,E200,E200,2023-03-14 08:47:15,Me ei tohi väsida Ukraina toetamisest!,tohi väsida ukraina toetamisest,Text(text='tohi väsida ukraina toetamisest'),tohtima väsima ukraina toetamine,ei tohi väsida ukraina toetamisest!
1,E200,E200,2023-02-24 12:25:09,Meie iseseisvuspäev ja Ukraina on kurval moel ...,iseseisvuspäev ukraina kurval moel seotud täna...,Text(text='iseseisvuspäev ukraina kurval moel ...,iseseisvuspäev ukraina kurb mood seotud täna m...,meie iseseisvuspäev ukraina kurval moel seotud...
2,E200,E200,2023-02-24 00:55:51,105 aastat tagasi algas Eesti ja aasta tagasi ...,aastat eesti aasta ukraina teekond euroopasse ...,Text(text='aastat eesti aasta ukraina teekond ...,aasta eesti aasta ukraina teekond euroopasse e...,105 aastat tagasi algas eesti aasta tagasi ukr...
3,E200,E200,2023-02-21 17:36:03,"Vladimir Putini tänane suur kõne, mis pidi löö...",putini tänane suur kõne lööma laineid juhtima ...,Text(text='putini tänane suur kõne lööma laine...,putin tänane suur kõne lööma laine juhtima täh...,"putini tänane suur kõne, mis pidi lööma lainei..."
4,E200,E200,2023-02-13 10:06:46,"Sinu kandidaadid Hiiu-, Lääne- ja Saaremaal. T...",kandidaadid hiiu lääne saaremaal tutvu lähemal...,Text(text='kandidaadid hiiu lääne saaremaal tu...,kandidaat hiid lääs saaremaa tutvuma lähemalt ...,"sinu kandidaadid hiiu -, lääne - saaremaal. tu..."


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3223 entries, 0 to 3222
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   nimi          3223 non-null   object        
 1   erakond       3223 non-null   object        
 2   kuupäev       3223 non-null   datetime64[ns]
 3   postitus      3223 non-null   object        
 4   clean_text    3223 non-null   object        
 5   estnltk_text  3223 non-null   object        
 6   lemma_text    3223 non-null   object        
 7   bert_text     3223 non-null   object        
dtypes: datetime64[ns](1), object(7)
memory usage: 201.6+ KB


# BERT

- https://huggingface.co/blog/sentiment-analysis-python
- https://huggingface.co/docs/transformers/main_classes/pipelines

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('tartuNLP/EstBERT')

In [ ]:
 #model = AutoModelForSequenceClassification.from_pretrained('bertSentiment',local_files_only=True)
 model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/MSc/bertSentiment/',local_files_only=True)

In [ ]:
 sentiment_model = pipeline(task = 'sentiment-analysis', model = model, tokenizer = tokenizer)

In [ ]:
%%time

import textwrap


sentiment_bert = []

for text in df['bert_text']:
    if len(text) > 512:
        split_into = math.ceil(len(text) / 512)
        split_len = math.ceil(len(text) / split_into)
        text_splits = textwrap.wrap(text, split_len)
        results = 0
        for split in text_splits:
            split_sentiment = sentiment_model([split])
            result_number = split_sentiment[0]['label'].split("_")[1]
            results += int(result_number)
            
        final_label = 'LABEL_' + str(round(results / len(text_splits)))
        sentiment_bert.append(final_label)
    else:
        sentiment = sentiment_model([text])
        sentiment_bert.append(sentiment[0]['label'])

CPU times: user 34min 24s, sys: 2.69 s, total: 34min 27s
Wall time: 34min 47s


In [ ]:
from collections import Counter
Counter(sentiment_bert)

Counter({'LABEL_0': 1222, 'LABEL_2': 983, 'LABEL_1': 1018})

In [ ]:
# add sentiments to df
df['sentiment_bert'] = sentiment_bert
df

In [ ]:
# save file
df.to_excel('leaders_sentiment_bert.xlsx', index = False)